In [26]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

## proccesing using requests

In [2]:
import requests
import json
import pandas as pd

# get a list of all manga
base_url = "https://api.mangadex.org"
uuid = "a990c2a1-b54f-4f4f-986c-3cdab1c9f2f9"
r = requests.get(f"{base_url}/chapter?manga={uuid}")

In [4]:
pd.DataFrame(r.json()).head()

,result,response,data,limit,offset,total
0,ok,collection,"{'id': '1ab38d10-5a67-42e2-80c9-4c5e99e79345',...",10,0,754
1,ok,collection,"{'id': 'c2d304aa-135f-4de7-8526-d4b73c3114fc',...",10,0,754
2,ok,collection,"{'id': 'cf65eca6-a331-4dcf-b3de-2f9756db55f8',...",10,0,754
3,ok,collection,"{'id': 'fa3600ee-8b9f-4d7a-ba94-7d1d414c8d55',...",10,0,754
4,ok,collection,"{'id': 'c4c2594f-2ed2-4480-b147-56341a7b9edd',...",10,0,754


In [8]:
pd.DataFrame(r.json()["data"])

,id,type,attributes,relationships
0,1ab38d10-5a67-42e2-80c9-4c5e99e79345,chapter,"{'volume': '1', 'chapter': '1', 'title': 'Let'...",[{'id': 'df075688-f7da-4cd0-9805-90ec13b48b3f'...
1,c2d304aa-135f-4de7-8526-d4b73c3114fc,chapter,"{'volume': '1', 'chapter': '2', 'title': 'Let'...",[{'id': 'df075688-f7da-4cd0-9805-90ec13b48b3f'...
2,cf65eca6-a331-4dcf-b3de-2f9756db55f8,chapter,"{'volume': '1', 'chapter': '3', 'title': 'Let'...",[{'id': 'df075688-f7da-4cd0-9805-90ec13b48b3f'...
3,fa3600ee-8b9f-4d7a-ba94-7d1d414c8d55,chapter,"{'volume': '1', 'chapter': '4', 'title': 'Let'...",[{'id': 'df075688-f7da-4cd0-9805-90ec13b48b3f'...
4,c4c2594f-2ed2-4480-b147-56341a7b9edd,chapter,"{'volume': '1', 'chapter': '5', 'title': 'Let'...",[{'id': 'df075688-f7da-4cd0-9805-90ec13b48b3f'...
5,6d42fdf2-9204-4812-8fbe-be2e4e548bf2,chapter,"{'volume': '1', 'chapter': '6', 'title': 'Let'...",[{'id': 'df075688-f7da-4cd0-9805-90ec13b48b3f'...
6,f65e6506-357c-4919-bee0-e53de3778152,chapter,"{'volume': '1', 'chapter': '7', 'title': 'Let'...",[{'id': 'df075688-f7da-4cd0-9805-90ec13b48b3f'...
7,275c4f28-508c-4b76-bf8c-59c3827be711,chapter,"{'volume': '1', 'chapter': '7.5', 'title': 'Om...",[{'id': 'efd8e5ca-eef9-43c6-9a1a-d5199c43d758'...
8,d8c15f44-c61e-434d-ba8b-e27c7fbb2453,chapter,"{'volume': '2', 'chapter': '8', 'title': 'Let'...",[{'id': 'df075688-f7da-4cd0-9805-90ec13b48b3f'...
9,62655be8-16a8-4646-ac0a-7fbeae4a4dd4,chapter,"{'volume': '2', 'chapter': '9', 'title': 'Let'...",[{'id': 'df075688-f7da-4cd0-9805-90ec13b48b3f'...


In [9]:
# and just testing around for the scraper offset logic
list(range(100, 350, 100))

[100, 200, 300]

## processing scraped data with spark

In [4]:
from pyspark.sql import functions as F
from manga_recsys.spark import get_spark

spark = get_spark()
df = spark.read.json("../data/raw/2022-12-16-mangadex-chapter.ndjson")
df.printSchema()
df.show(n=3, truncate=60, vertical=True)

root
 |-- attributes: struct (nullable = true)
 |    |-- chapter: string (nullable = true)
 |    |-- createdAt: string (nullable = true)
 |    |-- externalUrl: string (nullable = true)
 |    |-- pages: long (nullable = true)
 |    |-- publishAt: string (nullable = true)
 |    |-- readableAt: string (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- translatedLanguage: string (nullable = true)
 |    |-- updatedAt: string (nullable = true)
 |    |-- version: long (nullable = true)
 |    |-- volume: string (nullable = true)
 |-- id: string (nullable = true)
 |-- relationships: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)

-RECORD 0---------------------------------------------------------------------
 attributes    | {1, 2019-06-17T18:50:42+00:00, null, 31, 2019-06-17T18:50... 
 id            | 6e404a94-3624-4ded-8f2a-f3

In [5]:
df.count()

1488685

In [7]:
df.select("attributes.*").show(n=1, truncate=60, vertical=True)

-RECORD 0---------------------------------------
 chapter            | 1                         
 createdAt          | 2019-06-17T18:50:42+00:00 
 externalUrl        | null                      
 pages              | 31                        
 publishAt          | 2019-06-17T18:50:42+00:00 
 readableAt         | 2019-06-17T18:50:42+00:00 
 title              |                           
 translatedLanguage | en                        
 updatedAt          | 2019-06-17T18:50:42+00:00 
 version            | 1                         
 volume             | null                      
only showing top 1 row



In [11]:
df.select("relationships").show(n=1, truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------
 relationships | [{0058de88-ef1e-4257-ae8b-9d33e7a07636, scanlation_group}, {84c7d300-80fc-4f31-99da-5833ebdc842a, manga}, {a35c2b5d-c290-4024-b4ae-de1452ff291a, user}] 
only showing top 1 row



In [19]:
translated_counts = (
    df.select("id", "attributes.translatedLanguage", F.explode("relationships"))
    .select(
        "id", "translatedLanguage", F.col("col.id").alias("relationship_id"), "col.type"
    )
    .where("type == 'manga'")
    .groupBy("translatedLanguage")
    .agg(
        F.countDistinct("relationship_id").alias("countManga"),
        F.count("*").alias("countChapter"),
    )
    .orderBy("countManga", ascending=False)
)
translated_counts.show()

+------------------+----------+------------+
|translatedLanguage|countManga|countChapter|
+------------------+----------+------------+
|                en|     40139|      716144|
|             pt-br|      6773|      178521|
|                fr|      4694|       86084|
|             es-la|      4390|       74189|
|                ru|      4160|      117129|
|                pl|      3883|       72745|
|                id|      3499|       34818|
|                tr|      2791|       51149|
|                it|      2635|       43410|
|                vi|      1459|       14310|
|                es|      1400|       31987|
|             zh-hk|       774|       10517|
|                de|       544|        6774|
|                ar|       533|       11159|
|                hu|       499|        9903|
|                zh|       450|        6065|
|                uk|       419|        5676|
|                th|       418|        3819|
|                bg|       247|        3317|
|         

In [22]:
translated_counts.toPandas().fillna("unknown").to_json(
    "../data/processed/2022-12-16-mangadex-chapter-translated-counts.json",
    orient="records",
)

In [25]:
df.select(F.explode("relationships")).select("col.*").groupBy("type").count().show()

+----------------+-------+
|            type|  count|
+----------------+-------+
|scanlation_group|1533191|
|            user|1486386|
|           manga|1488685|
+----------------+-------+



In [31]:
from manga_recsys.commands.utils import mappify_struct

df.select(mappify_struct("relationships").alias("relationships")).show(
    truncate=False, n=3
)

+----------------------------------------------------------+
|relationships                                             |
+----------------------------------------------------------+
|{type -> user, id -> a35c2b5d-c290-4024-b4ae-de1452ff291a}|
|{type -> user, id -> a35c2b5d-c290-4024-b4ae-de1452ff291a}|
|{type -> user, id -> a35c2b5d-c290-4024-b4ae-de1452ff291a}|
+----------------------------------------------------------+
only showing top 3 rows



In [38]:
from manga_recsys.commands.utils import pivot_relationship

pivoted = pivot_relationship(df, "relationships")
pivoted.printSchema()
pivoted.show(truncate=False, n=3, vertical=True)

root
 |-- id: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- chapter: string (nullable = true)
 |    |-- createdAt: string (nullable = true)
 |    |-- externalUrl: string (nullable = true)
 |    |-- pages: long (nullable = true)
 |    |-- publishAt: string (nullable = true)
 |    |-- readableAt: string (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- translatedLanguage: string (nullable = true)
 |    |-- updatedAt: string (nullable = true)
 |    |-- version: long (nullable = true)
 |    |-- volume: string (nullable = true)
 |-- type: string (nullable = true)
 |-- relationships: struct (nullable = true)
 |    |-- manga: string (nullable = true)
 |    |-- scanlation_group: string (nullable = true)
 |    |-- user: string (nullable = true)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 id            | 15508